In [1]:
import pickle

In [2]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_FilmTrust'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/filmtrust'

In [3]:
MYMEDIALITEPATH='/home/kirana/MyMediaLite/bin'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pickle
from sklearn import metrics

In [5]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df_side_cold.pkl','rb'))

In [6]:
dftrain.reset_index(inplace=True)
dfvalid.reset_index(inplace=True)

In [7]:
dftrain['rating'].describe()

count    32036.000000
mean         2.999906
std          0.919053
min          0.500000
25%          2.500000
50%          3.000000
75%          4.000000
max          4.000000
Name: rating, dtype: float64

In [8]:
dfinp_train=dftrain[['userId','itemId','rating']]
dfinp_test=dfvalid[['userId','itemId','rating']]
dfinp_train.to_csv(f'{PATH}/inp_train_filmtrust.csv',index=False,header=False,sep=",")
dfinp_test.to_csv(f'{PATH}/inp_test_filmtrust.csv',index=False,header=False,sep=",")

In [9]:
TRAINFILEPATH=f'{PATH}/inp_train_filmtrust.csv'
TESTFILEPATH=f'{PATH}/inp_test_filmtrust.csv'

In [10]:
dfinp_train.head()

,userId,itemId,rating
0,871,6,2.5
1,959,2,3.5
2,111,2,0.5
3,693,215,3.5
4,672,1,4.0


### Global Average

In [11]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=GlobalAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_globalavg.txt'

In [12]:
!command {cmd}

=loading_time 0.05
memory 4
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
GlobalAverage  training_time 00:00:00.0003980 RMSE 0.9151019 MAE 0.7162019 CBD 0.2564884 new users: RMSE 1.02027 MAE 0.81573 CBD 0.26254 new items: RMSE 1.04138 MAE 0.831 CBD 0.27584  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:00.0105080
prediction_time 00:00:00.0042070

memory 4
>

In [13]:
predictions_globalavg=pd.read_csv (f'{DATAPATH}/prediction_globalavg.txt',sep='\t',header=None).iloc[:,2]

In [14]:
dfvalid['pred_globalavg']=predictions_globalavg

In [15]:
def score_by_group (dfvalid,predVar,groupVar='cold_start_group',method='global_avg'):
    scores=[]
    mae_scores=[]
    r2_scores=[]
    groups=dfvalid[groupVar].value_counts().index.tolist()
    for group in groups:
        df=dfvalid.loc[dfvalid[groupVar]==group]
        actual=df['rating']
        score=metrics.mean_squared_error(df['rating'],df[predVar])
        mae_score=metrics.mean_absolute_error(df['rating'],df[predVar])
        r2_score=metrics.r2_score(df['rating'],df[predVar])
        scores.append(score)
        mae_scores.append(mae_score)
        r2_scores.append(r2_score)
    return pd.DataFrame({'method':method, 'mse':scores, 'rmse':np.sqrt(scores), 'mae':mae_scores, 'r2_score':r2_scores, 'Cold Start Group':groups})

In [16]:
dfvalid['pred_globalavg']

0       2.999906
1       2.999906
2       2.999906
3       2.999906
4       2.999906
5       2.999906
6       2.999906
7       2.999906
8       2.999906
9       2.999906
10      2.999906
11      2.999906
12      2.999906
13      2.999906
14      2.999906
15      2.999906
16      2.999906
17      2.999906
18      2.999906
19      2.999906
20      2.999906
21      2.999906
22      2.999906
23      2.999906
24      2.999906
25      2.999906
26      2.999906
27      2.999906
28      2.999906
29      2.999906
          ...   
3428    2.999906
3429    2.999906
3430    2.999906
3431    2.999906
3432    2.999906
3433    2.999906
3434    2.999906
3435    2.999906
3436    2.999906
3437    2.999906
3438    2.999906
3439    2.999906
3440    2.999906
3441    2.999906
3442    2.999906
3443    2.999906
3444    2.999906
3445    2.999906
3446    2.999906
3447    2.999906
3448    2.999906
3449    2.999906
3450    2.999906
3451    2.999906
3452    2.999906
3453    2.999906
3454    2.999906
3455    2.9999

In [17]:
result_globalavg=pd.concat([score_by_group(dfvalid,'pred_globalavg'),score_by_group(dfvalid,'pred_globalavg','cold_start_bucket'),\
score_by_group(dfvalid,'pred_globalavg','flag_train')],axis=0)

In [18]:
result_globalavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,global_avg,0.816507,0.903608,0.706052,-0.001488,No-cold-start
1,global_avg,1.008096,1.004040,0.800394,-0.000160,User-Cold-Start
2,global_avg,0.933037,0.965939,0.758959,-0.000087,Item-Cold_start
3,global_avg,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,global_avg,0.816507,0.903608,0.706052,-0.001488,0
1,global_avg,1.023635,1.011749,0.806621,-0.000336,1
0,global_avg,0.837412,0.915102,0.716202,-0.001037,0


In [19]:
dfvalid['flag_train'].value_counts()

0    3458
Name: flag_train, dtype: int64

### User Average

In [20]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_useravg.txt'

In [21]:
!command {cmd}

=loading_time 0.05
memory 4
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
UserAverage  training_time 00:00:00.0019470 RMSE 0.8377964 MAE 0.6539876 CBD 0.2442118 new users: RMSE 1.02027 MAE 0.81573 CBD 0.26254 new items: RMSE 1.06993 MAE 0.83139 CBD 0.27879  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:00.0101100
prediction_time 00:00:00.0043600

memory 4
>

In [22]:
predictions_useravg=pd.read_csv (f'{DATAPATH}/prediction_useravg.txt',sep='\t',header=None).iloc[:,2]

In [23]:
dfvalid['pred_useravg']=predictions_useravg

In [24]:
result_useravg=pd.concat([score_by_group(dfvalid,'pred_useravg','cold_start_group','useravg'),score_by_group(dfvalid,'pred_useravg','cold_start_bucket','useravg'),\
score_by_group(dfvalid,'pred_useravg','flag_train','useravg')],axis=0)
result_useravg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,useravg,0.664410,0.815114,0.636845,0.185066,No-cold-start
1,useravg,1.008096,1.004040,0.800394,-0.000160,User-Cold-Start
2,useravg,1.009449,1.004714,0.759445,-0.081990,Item-Cold_start
3,useravg,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,useravg,0.664410,0.815114,0.636845,0.185066,0
1,useravg,1.035896,1.017790,0.806699,-0.012317,1
0,useravg,0.701903,0.837796,0.653988,0.160950,0


### Item Average

In [25]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemavg.txt'

In [26]:
!command {cmd}

=loading_time 0.05
memory 4
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
ItemAverage  training_time 00:00:00.0020110 RMSE 0.915014 MAE 0.7226846 CBD 0.2598779 new users: RMSE 0.99029 MAE 0.79064 CBD 0.25882 new items: RMSE 1.04138 MAE 0.831 CBD 0.27584  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:00.0100850
prediction_time 00:00:00.0044460

memory 4
>

In [27]:
predictions_itemavg=pd.read_csv (f'{DATAPATH}/prediction_itemavg.txt',sep='\t',header=None).iloc[:,2]

In [28]:
dfvalid['pred_itemavg']=predictions_itemavg

In [29]:
result_itemavg=pd.concat([score_by_group(dfvalid,'pred_itemavg','cold_start_group','itemavg'),score_by_group(dfvalid,'pred_itemavg','cold_start_bucket','itemavg'),\
score_by_group(dfvalid,'pred_itemavg','flag_train','itemavg')],axis=0)
result_itemavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemavg,0.822009,0.906647,0.715627,-0.008237,No-cold-start
1,itemavg,0.944563,0.971886,0.773948,0.062873,User-Cold-Start
2,itemavg,0.933037,0.965939,0.758959,-0.000087,Item-Cold_start
3,itemavg,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,itemavg,0.822009,0.906647,0.715627,-0.008237,0
1,itemavg,0.973028,0.986422,0.785555,0.049120,1
0,itemavg,0.837251,0.915014,0.722685,-0.000844,0


## Neighborhood Methods

### Item KNN Cosine

In [30]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemknn_cosine.txt'

In [31]:
!command {cmd}

=loading_time 0.06
memory 5
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
ItemKNN k=80 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=15 reg_i=10 num_iter=10  training_time 00:00:04.3017970 RMSE 0.8137834 MAE 0.62903 CBD 0.2409023 new users: RMSE 0.98271 MAE 0.79327 CBD 0.25584 new items: RMSE 1.06012 MAE 0.84509 CBD 0.27826  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:05.6786770
prediction_time 00:00:05.6079640

memory 28
>

In [32]:
predictions_itemknn_cosine=pd.read_csv (f'{DATAPATH}/prediction_itemknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_cosine']=predictions_itemknn_cosine

In [33]:
result_itemknncosine=pd.concat([score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_group','itemknncosine'),score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_bucket','itemknncosine'),\
score_by_group(dfvalid,'pred_itemknn_cosine','flag_train','itemknncosine')],axis=0)
result_itemknncosine

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemknncosine,0.627865,0.792379,0.610890,0.229891,No-cold-start
1,itemknncosine,0.928814,0.963750,0.776717,0.078498,User-Cold-Start
2,itemknncosine,0.982966,0.991446,0.776825,-0.053604,Item-Cold_start
3,itemknncosine,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,itemknncosine,0.627865,0.792379,0.610890,0.229891,0
1,itemknncosine,0.968494,0.984121,0.790628,0.053550,1
0,itemknncosine,0.662243,0.813783,0.629030,0.208358,0


### Item KNN Pearson

In [34]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemknn_pearson.txt'

In [35]:
!command {cmd}

=loading_time 0.06
memory 5
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
ItemKNN k=80 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=15 reg_i=10 num_iter=10  training_time 00:00:04.3715840 RMSE 0.8137834 MAE 0.62903 CBD 0.2409023 new users: RMSE 0.98271 MAE 0.79327 CBD 0.25584 new items: RMSE 1.06012 MAE 0.84509 CBD 0.27826  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:05.7521500
prediction_time 00:00:05.8886550

memory 28
>

In [36]:
predictions_itemknn_pearson=pd.read_csv (f'{DATAPATH}/prediction_itemknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_pearson']=predictions_itemknn_pearson

In [37]:
result_itemknnpearson=pd.concat([score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_group','itemknnpearson'),score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_bucket','itemknnpearson'),\
score_by_group(dfvalid,'pred_itemknn_pearson','flag_train','itemknnpearson')],axis=0)
result_itemknnpearson

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemknnpearson,0.627865,0.792379,0.610890,0.229891,No-cold-start
1,itemknnpearson,0.928814,0.963750,0.776717,0.078498,User-Cold-Start
2,itemknnpearson,0.982966,0.991446,0.776825,-0.053604,Item-Cold_start
3,itemknnpearson,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,itemknnpearson,0.627865,0.792379,0.610890,0.229891,0
1,itemknnpearson,0.968494,0.984121,0.790628,0.053550,1
0,itemknnpearson,0.662243,0.813783,0.629030,0.208358,0


### User KNN Pearson

In [38]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_userknn_pearson.txt '

In [39]:
!command {cmd}

=loading_time 0.06
memory 6
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
UserKNN k=80 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=15 reg_i=10 num_iter=10  training_time 00:00:03.6778520 RMSE 0.838982 MAE 0.6552835 CBD 0.2462157 new users: RMSE 0.98271 MAE 0.79327 CBD 0.25584 new items: RMSE 1.06012 MAE 0.84509 CBD 0.27826  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:09.3723960
prediction_time 00:00:09.2984770

memory 20
>

In [40]:
predictions_userknn_pearson=pd.read_csv (f'{DATAPATH}/prediction_userknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_pearson']=predictions_userknn_pearson

In [41]:
result_userknnpearson=pd.concat([score_by_group(dfvalid,'pred_userknn_pearson','cold_start_group','userknnpearson'),score_by_group(dfvalid,'pred_userknn_pearson','cold_start_bucket','userknnpearson'),\
score_by_group(dfvalid,'pred_userknn_pearson','flag_train','userknnpearson')],axis=0)
result_userknnpearson

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknnpearson,0.674188,0.821089,0.640090,0.173074,No-cold-start
1,userknnpearson,0.928814,0.963750,0.776717,0.078498,User-Cold-Start
2,userknnpearson,0.982966,0.991446,0.776825,-0.053604,Item-Cold_start
3,userknnpearson,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,userknnpearson,0.674188,0.821089,0.640090,0.173074,0
1,userknnpearson,0.968494,0.984121,0.790628,0.053550,1
0,userknnpearson,0.703891,0.838982,0.655283,0.158573,0


### User KNN Cosine

In [42]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_userknn_cosine.txt '

In [43]:
!command {cmd}

=loading_time 0.06
memory 6
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
UserKNN k=80 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=15 reg_i=10 num_iter=10  training_time 00:00:03.6840110 RMSE 0.838982 MAE 0.6552835 CBD 0.2462157 new users: RMSE 0.98271 MAE 0.79327 CBD 0.25584 new items: RMSE 1.06012 MAE 0.84509 CBD 0.27826  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:09.2442410
prediction_time 00:00:09.1908310

memory 20
>

In [44]:
predictions_userknn_cosine=pd.read_csv (f'{DATAPATH}/prediction_userknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_cosine']=predictions_userknn_cosine

In [45]:
result_userknncosine=pd.concat([score_by_group(dfvalid,'pred_userknn_cosine','cold_start_group','userknncosine'),score_by_group(dfvalid,'pred_userknn_cosine','cold_start_bucket','userknncosine'),\
score_by_group(dfvalid,'pred_userknn_cosine','flag_train','userknncosine')],axis=0)
result_userknncosine

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknncosine,0.674188,0.821089,0.640090,0.173074,No-cold-start
1,userknncosine,0.928814,0.963750,0.776717,0.078498,User-Cold-Start
2,userknncosine,0.982966,0.991446,0.776825,-0.053604,Item-Cold_start
3,userknncosine,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,userknncosine,0.674188,0.821089,0.640090,0.173074,0
1,userknncosine,0.968494,0.984121,0.790628,0.053550,1
0,userknncosine,0.703891,0.838982,0.655283,0.158573,0


## Matrix Factorization

### SVD

In [46]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=MatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_svd.txt'

In [47]:
!command {cmd}

=loading_time 0.05
memory 4
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
MatrixFactorization num_factors=10 regularization=0.015 learn_rate=0.01 learn_rate_decay=1 num_iter=30  training_time 00:00:00.2183470 RMSE 0.8473524 MAE 0.6516843 CBD 0.2468176 new users: RMSE 1.02027 MAE 0.81573 CBD 0.26254 new items: RMSE 1.04138 MAE 0.831 CBD 0.27584  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:00.0081430
prediction_time 00:00:00.0046410

memory 5
>

In [48]:
predictions_svd=pd.read_csv (f'{DATAPATH}/prediction_svd.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svd']=predictions_svd

In [49]:
result_svd=pd.concat([score_by_group(dfvalid,'pred_svd','cold_start_group','svd'),score_by_group(dfvalid,'pred_svd','cold_start_bucket','svd'),\
score_by_group(dfvalid,'pred_svd','flag_train','svd')],axis=0)
result_svd

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svd,0.683698,0.826860,0.634292,0.161409,No-cold-start
1,svd,1.008096,1.004040,0.800394,-0.000160,User-Cold-Start
2,svd,0.933037,0.965939,0.758959,-0.000087,Item-Cold_start
3,svd,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,svd,0.683698,0.826860,0.634292,0.161409,0
1,svd,1.023635,1.011749,0.806621,-0.000336,1
0,svd,0.718006,0.847352,0.651684,0.141700,0


### SVD Plus Plus

In [50]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=SVDPlusPlus \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_svdplusplus.txt '



In [51]:
cmd

'/home/kirana/MyMediaLite/bin/rating_prediction --training-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_FilmTrust/inp_train_filmtrust.csv --recommender=SVDPlusPlus --test-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_FilmTrust/inp_test_filmtrust.csv --prediction-file /home/kirana/Documents/final_dissertation_final/experiments/datasets/filmtrust/prediction_svdplusplus.txt '

In [52]:
!command {cmd}

=loading_time 0.05
memory 4
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
SVDPlusPlus num_factors=10 regularization=0.015 bias_reg=0.33 frequency_regularization=False learn_rate=0.001 bias_learn_rate=0.7 learn_rate_decay=1 num_iter=30  training_time 00:00:05.5479390 RMSE 0.8254516 MAE 0.6589829 CBD 0.2426074 new users: RMSE 0.98784 MAE 0.7973 CBD 0.25686 new items: RMSE 1.05327 MAE 0.8426 CBD 0.27741  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:00.0097460
prediction_time 00:00:00.0045370

memory 6
>

In [53]:
predictions_svdplusplus=pd.read_csv (f'{DATAPATH}/prediction_svdplusplus.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svdplusplus']=predictions_svdplusplus

result_svdplusplus=pd.concat([score_by_group(dfvalid,'pred_svdplusplus','cold_start_group','svdplusplus'),score_by_group(dfvalid,'pred_svdplusplus','cold_start_bucket','svdplusplus'),\
score_by_group(dfvalid,'pred_svdplusplus','flag_train','svdplusplus')],axis=0)
result_svdplusplus

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svdplusplus,0.648519,0.805307,0.643882,0.204558,No-cold-start
1,svdplusplus,0.939454,0.969254,0.780966,0.067942,User-Cold-Start
2,svdplusplus,0.964594,0.982138,0.773659,-0.033912,Item-Cold_start
3,svdplusplus,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,svdplusplus,0.648519,0.805307,0.643882,0.204558,0
1,svdplusplus,0.974022,0.986925,0.793504,0.048149,1
0,svdplusplus,0.681370,0.825452,0.658983,0.185494,0


### BiasedMatrixFactorization

In [54]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=BiasedMatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_biasedmatrixfactorization.txt '



In [55]:
!command {cmd}

=loading_time 0.05
memory 4
ratings range: [0.5, 4]
training data: 1481 users, 2002 items, 32036 ratings, sparsity 98.91951
test data:     992 users, 560 items, 3458 ratings, sparsity 99.37752
BiasedMatrixFactorization num_factors=10 bias_reg=0.01 reg_u=0.015 reg_i=0.015 frequency_regularization=False learn_rate=0.01 bias_learn_rate=1 learn_rate_decay=1 num_iter=30 bold_driver=False loss=RMSE max_threads=1 naive_parallelization=False  training_time 00:00:00.2321020 RMSE 0.8198133 MAE 0.6393524 CBD 0.2409226 new users: RMSE 0.98114 MAE 0.78823 CBD 0.2556 new items: RMSE 1.07244 MAE 0.84397 CBD 0.2798  new users and items: RMSE 1.28447 MAE 1.09997 CBD 0.3394 testing_time 00:00:00.0086770
prediction_time 00:00:00.0043250

memory 5
>

In [56]:
predictions_biasedmatrixfactorization=pd.read_csv (f'{DATAPATH}/prediction_biasedmatrixfactorization.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_biasedmatrixfactorization']=predictions_biasedmatrixfactorization

result_biasedmatrixfactorization=pd.concat([score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_group','biasedmatrixfactorization'),score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_bucket','biasedmatrixfactorization'),\
score_by_group(dfvalid,'pred_biasedmatrixfactorization','flag_train','biasedmatrixfactorization')],axis=0)
result_biasedmatrixfactorization

,method,mse,rmse,mae,r2_score,Cold Start Group
0,biasedmatrixfactorization,0.638513,0.799070,0.622871,0.216831,No-cold-start
1,biasedmatrixfactorization,0.925556,0.962058,0.771413,0.081731,User-Cold-Start
2,biasedmatrixfactorization,1.016274,1.008104,0.775404,-0.089305,Item-Cold_start
3,biasedmatrixfactorization,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,biasedmatrixfactorization,0.638513,0.799070,0.622871,0.216831,0
1,biasedmatrixfactorization,0.971243,0.985517,0.786175,0.050864,1
0,biasedmatrixfactorization,0.672094,0.819813,0.639352,0.196583,0


In [57]:
results_concat=pd.concat ([result_biasedmatrixfactorization,
result_svdplusplus,
result_svd,
result_userknncosine,
result_userknnpearson,
result_itemknnpearson,
result_itemknncosine,
result_itemavg,
result_useravg,
result_globalavg],axis=0)

In [58]:
pickle.dump(results_concat,open(f'{DATAPATH}/diff_results.pkl','wb'))

In [59]:
results_concat

,method,mse,rmse,mae,r2_score,Cold Start Group
0,biasedmatrixfactorization,0.638513,0.799070,0.622871,0.216831,No-cold-start
1,biasedmatrixfactorization,0.925556,0.962058,0.771413,0.081731,User-Cold-Start
2,biasedmatrixfactorization,1.016274,1.008104,0.775404,-0.089305,Item-Cold_start
3,biasedmatrixfactorization,1.649868,1.284472,1.099969,-0.422300,User-Item-cold-start
0,biasedmatrixfactorization,0.638513,0.799070,0.622871,0.216831,0
1,biasedmatrixfactorization,0.971243,0.985517,0.786175,0.050864,1
0,biasedmatrixfactorization,0.672094,0.819813,0.639352,0.196583,0
0,svdplusplus,0.648519,0.805307,0.643882,0.204558,No-cold-start
1,svdplusplus,0.939454,0.969254,0.780966,0.067942,User-Cold-Start
2,svdplusplus,0.964594,0.982138,0.773659,-0.033912,Item-Cold_start


In [60]:
pickle.dump([dftrain,dfvalid,results_concat],open(f'{DATAPATH}/df_side_cold_runother.pkl','wb'))